In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("yellow_tripdata_2021-01.csv", nrows=100)

In [ ]:
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])

df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

In [5]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object

In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1,2.10,1,N,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1,0.20,1,N,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1,14.70,1,N,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0,10.60,1,N,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1,4.94,1,N,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5


In [7]:
print(pd.io.sql.get_schema(df, "yellowtaxi"))

CREATE TABLE "yellowtaxi" (
"VendorID" INTEGER,
  "tpep_pickup_datetime" TIMESTAMP,
  "tpep_dropoff_datetime" TIMESTAMP,
  "passenger_count" INTEGER,
  "trip_distance" REAL,
  "RatecodeID" INTEGER,
  "store_and_fwd_flag" TEXT,
  "PULocationID" INTEGER,
  "DOLocationID" INTEGER,
  "payment_type" INTEGER,
  "fare_amount" REAL,
  "extra" REAL,
  "mta_tax" REAL,
  "tip_amount" REAL,
  "tolls_amount" REAL,
  "improvement_surcharge" REAL,
  "total_amount" REAL,
  "congestion_surcharge" REAL
)


In [8]:
!pip list| grep SQL


SQLAlchemy                2.0.28


In [9]:
from sqlalchemy import create_engine

In [11]:
user = "root"
password = "root"
host = "localhost"
database = "ny_taxi"
port = "5432"

In [14]:
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

In [17]:
print(pd.io.sql.get_schema(df, "yellowtaxi", con=engine))


CREATE TABLE yellowtaxi (
	"VendorID" BIGINT, 
	tpep_pickup_datetime TIMESTAMP WITHOUT TIME ZONE, 
	tpep_dropoff_datetime TIMESTAMP WITHOUT TIME ZONE, 
	passenger_count BIGINT, 
	trip_distance FLOAT(53), 
	"RatecodeID" BIGINT, 
	store_and_fwd_flag TEXT, 
	"PULocationID" BIGINT, 
	"DOLocationID" BIGINT, 
	payment_type BIGINT, 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53), 
	congestion_surcharge FLOAT(53)
)




In [18]:
df_iter = pd.read_csv("yellow_tripdata_2021-01.csv", iterator=True, chunksize=100000)

In [19]:
df = next(df_iter)

In [22]:
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])

df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

In [27]:
df.head(n=0).to_sql(name="yellow_taxi_data", con=engine, if_exists="replace")

0

In [30]:
from time import time

In [31]:
while True:
    start_time = time()
    df = next(df_iter)

    df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
    df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

    df.to_sql(name="yellow_taxi_data", con=engine, if_exists="append")

    end_time = time()

    print(f"inserted one chunk... took {end_time-start_time} seconds")

inserted one chunk... took 9.565128564834595 seconds
inserted one chunk... took 9.590503454208374 seconds
inserted one chunk... took 9.567494630813599 seconds
inserted one chunk... took 9.305852174758911 seconds
inserted one chunk... took 9.441633939743042 seconds
inserted one chunk... took 9.443900346755981 seconds
inserted one chunk... took 9.34519100189209 seconds
inserted one chunk... took 9.787870645523071 seconds
inserted one chunk... took 9.65328860282898 seconds
inserted one chunk... took 9.407649517059326 seconds
inserted one chunk... took 9.48084282875061 seconds


/tmp/ipykernel_4436/3130407715.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = next(df_iter)


inserted one chunk... took 10.058899879455566 seconds
inserted one chunk... took 6.133341550827026 seconds


StopIteration: 